# 解释器模式

**[参考链接]**
- http://blog.csdn.net/lovelion/article/details/17517213 中解释器模式部分

**定义:**

解释器模式(Interpreter Pattern): 定义一个语言的文法，并且建立一个解释器来解释该语言中的句子，这里的“语言”是指使用规定格式和语法的代码

**NOTE:**

- 表达式可分为终结符表达式和非终结符表达式，因此解释器模式的结构与组合模式的结构有些类似
- 在解释器模式中，每一种终结符和非终结符都有一个具体类与之对应，正因为使用类来表示每一条文法规则，所以系统将具有较好的灵活性和可扩展性

**角色:**

- AbstractExpression(抽象表达式): 声明抽象解释操作
- TerminalExpression(终结符表达式): 对终结符元素的处理
- NonterminalExpression(非终结符表达式): 可以包含终结符表达式和/或非终结符表达式
- Context(环境类): 环境类`Context`用于存储解释器之外的一些全局信息，它通常作为参数被传递到所有表达式的解释方法`interpret()`中，可以在`Context`对象中存储和访问表达式解释器的状态，向表达式解释器提供一些全局的、公共的数据，此外还可以在`Context`中增加一些所有表达式解释器都共有的功能，减轻解释器的职责

**结构图:**

![](https://github.com/luog1992/fs/blob/master/2017110400.png?raw=true)

## 示例1: 机器人控制指令

输入:

`up move 5 and left run 10 and down move 2`

输出:

向上移动5 再 向左快速移动10 再 向下移动2

文法规则:

```
expression ::= direction action distance | composite 	# 表达式
composite ::= expression 'and' expression 		# 复合表达式
direction ::= 'up' | 'down' | 'left' | 'right' 		# 移动方向
action ::= 'move' | 'run' 				# 移动方式
distance ::= an integer 				# 移动距离
```

In [6]:
# abstract expression
class AbstractNote(object):
    def interpret(self):
        raise NotImplementedError


# nonterminal expression: and
class AndNode(AbstractNote):
    def __init__(self, left, right):
        self.left = left
        self.right = right

    def interpret(self):
        # 非终结符表达式递归处理组成部分
        return self.left.interpret() + ' 再 ' + self.right.interpret()


# nonterminal expression: sentence
class SentenceNode(AbstractNote):
    def __init__(self, direction, action, distance):
        self.direction = direction
        self.action = action
        self.distance = distance

    def interpret(self):
        # 非终结符表达式递归处理组成部分
        return self.direction.interpret() + \
               self.action.interpret() + \
               self.distance.interpret()


# terminal expression: direction
class DirectionNode(AbstractNote):
    def __init__(self, direction):
        self.direction = direction.lower()

    def interpret(self):
        d = dict(
            up='向上',
            down='向下',
            left='向左',
            right='向右',
        )
        return d.get(self.direction, '无效指令')


# terminal expression: action
class ActionNode(AbstractNote):
    def __init__(self, action):
        self.action = action.lower()

    def interpret(self):
        d = dict(
            run='快速移动',
            move='移动',
        )
        return d.get(self.action, '无效指令')


# terminal expression: distance
class DistanceNode(AbstractNote):
    def __init__(self, distance):
        self.distnace = distance

    def interpret(self):
        return self.distnace

In [7]:
# 指令处理类, 用于将输入的语句指令转换为`expression`
class InstructionHandler(object):
    def __init__(self, instruction):
        self.instruction = instruction
        self.abstract_node = self.process(instruction)

    def process(self, instruction):
        words = [x.strip().lower() for x in instruction.split(' ') if x.strip()]

        i = 0
        nodes = []
        while i < len(words):
            if words[i] == 'and':
                left = nodes.pop()
                # 将and之后的三个word组成一个SentenceNode
                right = self.make_sentence(words[i+1], words[i+2], words[i+3])
                # 组装一个AndNode
                nodes.append(AndNode(left, right))
                i += 4
            else:
                # 将连续三个word组装为一个SentenceNode
                sentence = self.make_sentence(words[i], words[i+1], words[i+2])
                nodes.append(sentence)
                i += 3

        return nodes.pop()

    def make_sentence(self, direction, action, distance):
            direction = DirectionNode(direction)
            action = ActionNode(action)
            distance = DistanceNode(distance)
            return SentenceNode(direction, action, distance)

    def start(self):
        if self.abstract_node:
            return self.abstract_node.interpret()
        return 'No instructions'

In [11]:
instruction = 'up move 5 and left run 10 and down move 2'
handler = InstructionHandler(instruction)
result = handler.start()
print result

向上移动5 再 向左快速移动10 再 向下移动2


## 示例2: 字符格式化指令

例如将:

`loop 2 print PYTHON space space print JAVA break end print JAVASCRIPT space print RUBY`

输出为:
```
PYTHON  JAVA

PYTHON  JAVA

JAVASCRIPT RUBY
```

文法规则:
```
expression ::= command* 			# 表达式，一个表达式包含多条命令
command ::= loop | primitive 			# 语句命令
loop ::= 'loopnumber' expression  'end' 	# 循环命令，其中number为自然数
primitive ::= 'printstring'  | 'space' | 'break' 	# 基本命令，其中string为字符串
```

In [45]:
# 环境类: 存储和操作需要解释的语句
class Context(object):
    def __init__(self, text):
        self.text = text
        self.words = [x.strip() for x in text.split(' ') if x.strip()]
        assert self.words
        self.token_iter = iter(self.words)
        self.curr_token = self.token_iter.next()

    def move_on(self):
        # 移动到下一个标记
        try:
            self.curr_token = self.token_iter.next()
        except StopIteration:
            self.curr_token = None
        finally:
            return self.curr_token

    def skip_token(self, token):
        # 跳过当前标记
        if token != self.curr_token:
            print 'Error token %s != %s' % (token, self.curr_token)
        self.move_on()

    @property
    def curr_num(self):
        # 当前标记是数字, 返回之
        try:
            return int(self.curr_token)
        except ValueError:
            print 'Error %s is not a integer' % self.curr_token
            return 0


# 抽象节点类: 抽象表达式
class Node(object):
    def interpret(self, context):
        raise NotImplementedError

    def execute(self):
        raise NotImplementedError


# 表达式节点类: 非终结符表达式
class ExpressionNode(Node):
    def __init__(self):
        self.cmd_nodes = []

    def interpret(self, context):
        # 循环处理Context中的标记
        while True:
            if context.curr_token is None:
                break
            elif context.curr_token == 'end':  # 循环结束
                context.skip_token('end')
                break
            else:
                node = CommandNode()
                node.interpret(context)
                self.cmd_nodes.append(node)

    def execute(self):
        for node in self.cmd_nodes:
            node.execute()


# 语句命令节点类: 非终结符表达式
class CommandNode(Node):
    node = None

    def interpret(self, context):
        # 处理loop命令
        if context.curr_token == 'loop':
            self.node = LoopCommandNode()
            self.node.interpret(context)
        # 处理其他基本命令
        else:
            self.node = PrimitiveCommandNode()
            self.node.interpret(context)

    def execute(self):
        self.node.execute()


# 循环命令节点: 非终结符表达式
class LoopCommandNode(Node):
    number = 0
    node = None

    def interpret(self, context):
        context.skip_token('loop')
        self.number = context.curr_num
        context.move_on()
        # 循环语句中的表达式
        self.node = ExpressionNode()
        self.node.interpret(context)

    def execute(self):
        for i in range(self.number):
            self.node.execute()


# 基本命令节点类: 终结符表达式
class PrimitiveCommandNode(Node):
    name = None
    text = None

    def interpret(self, context):
        self.name = context.curr_token
        context.skip_token(self.name)
        if self.name not in ('print', 'break', 'space'):
            print 'Error 非法命令 %s' % self.name
        if self.name == 'print':
            self.text = context.curr_token
            context.move_on()

    def execute(self):
        if self.name == 'print':
            print self.text,
        elif self.name == 'space':
            print ' ',
        elif self.name == 'break':
            print '\n'

In [46]:
text = 'loop 2 print PYTHON space space print JAVA break end print JAVASCRIPT space print RUBY'
ctx = Context(text)
node = ExpressionNode()
node.interpret(ctx)
node.execute()

PYTHON     JAVA 

PYTHON     JAVA 

JAVASCRIPT   RUBY
